# Geographical Analysis of Brazilian E-Commerce

Olist has released a dataset with 100k orders made between 2016 and 2018. Each order has some information about the customer and contains the first three digits of the customer zip code. Olist has also released a geolocation database that has 323k lat/lng coordinates related to the first three digits of each zip code.

## CEP: the Brazilian Zip Code
A brazilian zip code, also know as CEP, stands for Postal Adressing Code (*Código de Endereçamento Postal*) and contains 8 digits. Introduced in 1972 as a sequence of five digits, it was expanded to eight digits in 1992 to allow for more precise localization. The standard format is "nnnnn-nnn" (the original five digits, an hyphen, and the new three digits).

**CEP:** 12.345-678

Most cities with population around 100,000 and above have a CEP assigned to every public place and to some high-occupancy private spaces, like major commercial buildings and large residential condos. Small towns are assigned a general 5-digit code followed by the suffix -000. 

* the first part is composed by 5 digits that represent Region, Subregion, Sector, Subsector and Subsector Splitter.
* the second part contain 3 digits, separated by an hyphen from the first, and it represents the Distribution Identifiers.

More info about how CEP works may be found at the [Correios website](https://www.correios.com.br/a-a-z/cep-codigo-de-enderecamento-postal).

Lets look at the geolocation dataset provided by Olist and try to understand how CEP works geographically.

In [ ]:
import numpy as np
import pandas as pd 
import os

geo = pd.read_csv("../input/geolocation_olist_public_dataset.csv")
geo.head(3)

There are 851 different zip_code_prefix. They are all limited to 500 samples per zip_code_prefix. On average there are 380 coordinates for each prefix.

In [ ]:
geo['zip_code_prefix'].value_counts().to_frame().describe()

There are some outliers coordinates in the dataset that are outside of brazilian territory. Lets guarantee that all coordinates are within a rectangle delimited by the limits of Brazil.

In [ ]:
# Removing some outliers
#Brazils most Northern spot is at 5 deg 16′ 27.8″ N latitude.;
geo = geo[geo.lat <= 5.27438888]
#it’s most Western spot is at 73 deg, 58′ 58.19″W Long.
geo = geo[geo.lng >= -73.98283055]
#It’s most southern spot is at 33 deg, 45′ 04.21″ S Latitude.
geo = geo[geo.lat >= -33.75116944]
#It’s most Eastern spot is 34 deg, 47′ 35.33″ W Long.
geo = geo[geo.lng <=  -34.79314722]

In [ ]:
from datashader.utils import lnglat_to_meters as webm
x, y = webm(geo['lng'], geo['lat'])
geo['x'] = pd.Series(x)
geo['y'] = pd.Series(y)

Then we treat the latitute and longitude coordinates and transform then to Mercator x/y Coordinates.

In [ ]:
geo.head(3)

## Zip Codes in Brazil
Finally plotting the coordinates on a map. We see there is a relationship between the zip code prefix and the location of that zip code. They start in Sao Paulo, with prefix 010, and then increase counterclockwise finishing in Rio Grande do Sul (south of Brazil), with prefix 999.

In [ ]:
brazil = geo
brazil['zip_code_prefix'].describe().to_frame()

In [ ]:
import holoviews as hv
import geoviews as gv
import datashader as ds
from colorcet import fire, rainbow
from holoviews.streams import RangeXY
from holoviews.operation.datashader import datashade, dynspread, rasterize
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
hv.extension('bokeh')

In [ ]:
%opts Overlay [width=700 height=450 toolbar='above' xaxis=None yaxis=None]
%opts QuadMesh [tools=['hover'] colorbar=True] (alpha=0 hover_alpha=0.2)

T = 0.3

def plot_map_min(data, label, agg_data):
    url="http://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{Z}/{Y}/{X}.png"
    geomap = gv.WMTS(url)
    points = hv.Points(gv.Dataset(data, kdims=['x', 'y'], vdims=[agg_data]))
    agg = datashade(points, element_type=gv.Image, aggregator=ds.min(agg_data), cmap=rainbow)
    zip_codes = dynspread(agg, threshold=T, max_px=2)
    hover = hv.util.Dynamic(rasterize(points, aggregator=ds.min(agg_data), width=40, height=20, streams=[RangeXY]), operation=hv.QuadMesh)
    hover = hover.options(cmap=rainbow)
    img = geomap * zip_codes * hover
    img = img.relabel(label)
    return img

In [ ]:
plot_map_min(brazil, 'Zip Codes in Brazil', 'zip_code_prefix')

## Zip Codes in States
Lets look at the state of Sao Paulo (SP) to see how zip code prefixes works in a regional level. We see that:
* zip code prefixes in Sao Paulo state ranges from 010 to 199
* zip codes starting with 0 are in the Sao Paulo metro region
* zip codes starting with 1 are in the interior of the state

In [ ]:
sp = geo[geo['state'] == 'sp']
#remove outliers
sp = sp[(sp.x <= sp.x.quantile(0.999)) & (sp.x >= sp.x.quantile(0.001))]
sp = sp[(sp.y <= sp.y.quantile(0.999)) & (sp.y >= sp.y.quantile(0.001))]
sp['zip_code_prefix'].describe().to_frame()

In [ ]:
plot_map_min(sp, 'Zip Codes in Sao Paulo State', 'zip_code_prefix')

## Zip Codes in Large Cities 
Lets look at the city of Sao Paulo to see how zip code prefixes works in a city level. We see that:
* zip code prefixes in Sao Paulo city ranges from 010 to 095
* zip code prefixes are somehow related to neighborhoods or city districts

In [ ]:
saopaulo = geo[geo['city'] == 'sao paulo']
#remove outliers
saopaulo = saopaulo[(saopaulo.x <= saopaulo.x.quantile(0.999)) & (saopaulo.x >= saopaulo.x.quantile(0.001))]
saopaulo = saopaulo[(saopaulo.y <= saopaulo.y.quantile(0.999)) & (saopaulo.y >= saopaulo.y.quantile(0.001))]
saopaulo['zip_code_prefix'].describe().to_frame()

In [ ]:
plot_map_min(saopaulo, 'Zip Codes in Sao Paulo City', 'zip_code_prefix')

## Zip Codes in Small Cities
Lets look at the city of Atibaia to see how zip code prefixes works in a city level. We see that:
* zip code prefix of Atibaia city is 129
* but there are other neighbor cities with the same zip code prefix
* to have more detail and go down to a city level we would probably need more zip code digits (the 4th and 5th digit)

In [ ]:
df = geo[geo['city'] == 'atibaia']
df['zip_code_prefix'].describe().to_frame()

In [ ]:
zip129 = geo[geo['zip_code_prefix'] == 129]
zip129[['zip_code_prefix', 'city', 'state']].drop_duplicates()

In [ ]:
%opts Overlay [width=700 height=450 toolbar='above' xaxis=None yaxis=None]
%opts QuadMesh [tools=['hover'] colorbar=True] (alpha=0 hover_alpha=0.2)

def plot_map_2(data, label):
    url="http://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{Z}/{Y}/{X}.png"
    geomap = gv.WMTS(url)
    points = hv.Points(gv.Dataset(data, kdims=['x', 'y'], vdims=['zip_code_prefix']))
    agg = datashade(points, element_type=gv.Image, aggregator=ds.min('zip_code_prefix'), cmap='blue')
    zip_codes = dynspread(agg, threshold=T, max_px=2)
    img = geomap * zip_codes
    img = img.relabel(label)
    return img

plot_map_2(zip129, 'Zip Code Prefix 129')

# Where does most revenue comes from?
Plotting the sum of products value grouped by zip code prefix we see that most of the revenue came from the Southeast and South regions of Brazil. It is also possible to see that large cities and capitals, where population is bigger, have larger participation on revenue. 

In [ ]:
orders = pd.read_csv('../input/olist_public_dataset_v2.csv')
brazil_geo = geo.set_index('zip_code_prefix').copy()

In [ ]:
gp = orders.groupby('customer_zip_code_prefix')['order_products_value'].sum().to_frame()
revenue = brazil_geo.join(gp)
revenue['revenue'] = revenue.order_products_value / 1000

In [ ]:
%opts Overlay [width=700 height=450 toolbar='above' xaxis=None yaxis=None]
%opts QuadMesh [tools=['hover'] colorbar=True] (alpha=0 hover_alpha=0.2)

def plot_map_max(data, label, agg_data):
    url="http://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{Z}/{Y}/{X}.png"
    geomap = gv.WMTS(url)
    points = hv.Points(gv.Dataset(data, kdims=['x', 'y'], vdims=[agg_data]))
    agg = datashade(points, element_type=gv.Image, aggregator=ds.max(agg_data), cmap=rainbow)
    zip_codes = dynspread(agg, threshold=T, max_px=2)
    hover = hv.util.Dynamic(rasterize(points, aggregator=ds.max(agg_data), width=40, height=20, streams=[RangeXY]), operation=hv.QuadMesh)
    hover = hover.options(cmap=rainbow)
    img = geomap * zip_codes * hover
    img = img.relabel(label)
    return img

In [ ]:
plot_map_max(revenue, 'Orders Revenue (thousands R$)', 'revenue')

# What is the Average Ticket?
Here we see something somehow unexpected. Customers of the south and southeast regions of Brazil have lower average ticket, than their peers on north and norteast. This might happen because they have to pay more for freight (let's check that in a moment)

In [ ]:
gp = orders.groupby('order_id').agg({'order_products_value': 'sum', 'customer_zip_code_prefix': 'max'})
gp = gp.groupby('customer_zip_code_prefix')['order_products_value'].mean().to_frame()
avg_ticket = brazil_geo.join(gp)
avg_ticket['avg_ticket'] = avg_ticket.order_products_value

In [ ]:
%opts Overlay [width=700 height=450 toolbar='above' xaxis=None yaxis=None]
%opts QuadMesh [tools=['hover'] colorbar=True] (alpha=0 hover_alpha=0.2)

def plot_map_mean(data, label, agg_data):
    url="http://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{Z}/{Y}/{X}.png"
    geomap = gv.WMTS(url)
    points = hv.Points(gv.Dataset(data, kdims=['x', 'y'], vdims=[agg_data]))
    agg = datashade(points, element_type=gv.Image, aggregator=ds.mean(agg_data), cmap=rainbow)
    zip_codes = dynspread(agg, threshold=T, max_px=2)
    hover = hv.util.Dynamic(rasterize(points, aggregator=ds.mean(agg_data), width=40, height=20, streams=[RangeXY]), operation=hv.QuadMesh)
    hover = hover.options(cmap=rainbow)
    img = geomap * zip_codes * hover
    img = img.relabel(label)
    return img

In [ ]:
plot_map_mean(avg_ticket, 'Orders Average Ticket (R$)', 'avg_ticket')

# Who pays more for transportation?
We might find a freight ratio by dividing the freight value by the order value. This ratio indicates the percentage of the product price that a person had to pay just to get their order delivered. For example, if a product costs R\$50.00 and the freight value was R\$10.00, then the freight ratio is 0.2 or 20%. Higher freight ratios are very likely to discourage customers to complete a purchase. Due to logistics costs, we expect to see lower freight ratios in densely populated areas and are higher freight ratios on sparsely poulated regions.

In [ ]:
gp = orders.groupby('order_id').agg({'order_products_value': 'sum', 'order_freight_value': 'sum', 'customer_zip_code_prefix': 'max'})
gp['freight_ratio'] = gp.order_freight_value / gp.order_products_value
gp = gp.groupby('customer_zip_code_prefix')['freight_ratio'].mean().to_frame()
freight_ratio = brazil_geo.join(gp)

In [ ]:
plot_map_mean(freight_ratio, 'Orders Average Freight Ratio', 'freight_ratio')

# Average Delivery Time
Unfortunately, who lives in the north and northeast of Brazil has to bear with higher freight costs and has to wait longer to receive their purchase.

In [ ]:
orders['order_delivered_customer_date'] = pd.to_datetime(orders.order_delivered_customer_date)
orders['order_aproved_at'] = pd.to_datetime(orders.order_aproved_at)
orders['actual_delivery_time'] = orders.order_delivered_customer_date - orders.order_aproved_at
orders['actual_delivery_time'] = orders['actual_delivery_time'].dt.days

In [ ]:
gp = orders.groupby('customer_zip_code_prefix')['actual_delivery_time'].mean().to_frame()
delivery_time = brazil_geo.join(gp)
delivery_time['avg_delivery_time'] = delivery_time['actual_delivery_time']

In [ ]:
plot_map_mean(delivery_time, 'Average Delivery Time (days)', 'avg_delivery_time')

## Interesting Point About Brazilian Suburbs
Unlike other countries, in Brazil the richest areas usualy are near downtow and suburbs are know for poverty and high violence rates. Lets explore that in Rio the Janeiro.

In [ ]:
riodejaneiro = geo[geo['city'] == 'rio de janeiro']
#remove outliers
riodejaneiro = riodejaneiro[(riodejaneiro.x <= riodejaneiro.x.quantile(0.999)) & (riodejaneiro.x >= riodejaneiro.x.quantile(0.001))]
riodejaneiro = riodejaneiro[(riodejaneiro.y <= riodejaneiro.y.quantile(0.999)) & (riodejaneiro.y >= riodejaneiro.y.quantile(0.001))]
riodejaneiro.set_index('zip_code_prefix', inplace=True)

In [ ]:
gp = orders.groupby('customer_zip_code_prefix')['actual_delivery_time'].mean().to_frame()
rj_delivery_time = riodejaneiro.join(gp)
rj_delivery_time['avg_delivery_time'] = rj_delivery_time['actual_delivery_time']

In [ ]:
plot_map_mean(rj_delivery_time, 'Average Delivery Time in Rio de Janeiro (days)', 'avg_delivery_time')

It turns out that if you live in rich neighborhoods such as Downtown, Botafogo, Copacabana and Flamengo you are likey to receive your order five days earlier than someone who lives in a poor neighborhood such as Cidade de Deus or Bangu. The same pattern may be observed on the average ticket: poor neighborhoods spend less on average.

In [ ]:
gp = orders.groupby('order_id').agg({'order_products_value': 'sum', 'customer_zip_code_prefix': 'max'})
gp = gp.groupby('customer_zip_code_prefix')['order_products_value'].mean().to_frame()
rj_avg_ticket = riodejaneiro.join(gp)
rj_avg_ticket['avg_ticket'] = rj_avg_ticket.order_products_value

In [ ]:
plot_map_mean(rj_avg_ticket, 'Orders Average Ticket in Rio de Janeiro(R$)', 'avg_ticket')

# Work in progress...

## To do:
1. Which categories are most sold.
2. Wich payment method was chosen. 
3. How many installments.
4. Analysis on specific cities, such as  Sao Paulo, Porto Alegre, Curitiba, Fortaleza, Bahia, Brasilia. 
5. Any sugestion?